In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import skimage
from PIL import Image
from sklearn.model_selection import train_test_split

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms
from torchvision.utils import make_grid, save_image

print('PyTorch version:', torch.__version__)
print('torchvision version:', torchvision.__version__)
use_gpu = torch.cuda.is_available()
print('Is GPU available:', use_gpu)

PyTorch version: 0.4.1
torchvision version: 0.2.1
Is GPU available: True


In [21]:
# general settings

# device
device = torch.device('cuda' if use_gpu else 'cpu')

# batchsize (same as AVID paper)
batchsize = 16

# seed setting (warning : cuDNN's randomness is remaining)
seed = 1
torch.manual_seed(seed)
if use_gpu:
    torch.cuda.manual_seed(seed)
    
# directory settings
# Data directory (for IR-MNIST)
data_dir = '../../data/IR-MNIST/'
train_data_dir = data_dir + 'Train_Samples/'
test_data_dir = data_dir + 'Test_Samples/'

# directory to put generated images
output_dir = data_dir + 'output/'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
# directory to save state_dict and loss.npy
save_dir = data_dir + 'save/'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

In [3]:
# make dataset class for image loading
class MyDataset(Dataset):
    def __init__(self, root_dir, transform = None):
        self.root_dir = root_dir
        self.list_dir = os.listdir(root_dir)
        self.transform = transform
        
    def __len__(self):
        return len(self.list_dir)
    
    def __getitem__(self, idx):
        img_name = self.root_dir + self.list_dir[idx]
        image = skimage.io.imread(img_name)
        
        if self.transform:
            image = self.transform(image)
            
        return image

In [4]:
# define transform
# Normalize [0~255] to [-1~1]
class Normalize:
    def __call__(self, image):
        return (image - 127.5) / 127.5
    
class Tofloat:
    def __call__(self, tensor):
        return tensor.float()
    
tf = transforms.Compose([Normalize(), transforms.ToTensor(), Tofloat()])

In [5]:
# make dataset
imgDataset = MyDataset(train_data_dir, transform = tf)

# split to train data and validation data
train_data, validation_data = train_test_split(imgDataset, test_size = 0.2, random_state = seed)

print('The number of training data:', len(train_data))
print('The number of validation data:', len(validation_data))

The number of training data: 4000
The number of validation data: 1000


In [42]:
# make DataLoader
train_loader = DataLoader(train_data, batch_size = batchsize, shuffle = True)
validation_loader = DataLoader(validation_data, batch_size = batchsize, shuffle = True)

In [25]:
# visualization of an example of training data
# comment out when runnnig in no GUI machine
'''
X = iter(train_loader).next()[0].numpy()
X = np.transpose(X, [1, 2, 0]) * 0.5 + 0.5
print(X.shape)
plt.imshow(X)
'''

'\nX = iter(train_loader).next()[0].numpy()\nX = np.transpose(X, [1, 2, 0]) * 0.5 + 0.5\nprint(X.shape)\nplt.imshow(X)\n'

In [8]:
# define parts for U-net and FCN for convenience
# downsampling
# conv > batchnorm > dropout > leakyrelu
class Downsample(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size = 4 , stride = 2, padding = 1, \
                                                                    use_batchnorm = True, use_dropout = False):
        super(Downsample, self).__init__()
        self.cv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.dr = nn.Dropout(0.5)
        self.rl = nn.LeakyReLU(0.2)
        
        self.use_batchnorm = use_batchnorm
        self.use_dropout = use_dropout
        
    def forward(self, x):
        out = self.cv(x)
        
        if self.use_batchnorm:
            out = self.bn(out)
            
        if self.use_dropout:
            out = self.dr(out)
            
        out = self.rl(out)
        
        return out

In [9]:
# define parts for U-net for convenience
# upsampling (using transposed convolution)
# conv > batchnorm > dropout > relu
class Upsample(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size = 4, stride = 2, padding = 1, \
                                                                   use_batchnorm = True, use_dropout = False):
        super(Upsample, self).__init__()
        self.tc = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.dr = nn.Dropout(0.5)
        self.rl = nn.ReLU()
        
        self.use_batchnorm = use_batchnorm
        self.use_dropout = use_dropout
        
    def forward(self, x):
        out = self.tc(x)
        
        if self.use_batchnorm:
            out = self.bn(out)
            
        if self.use_dropout:
            out = self.dr(out)
            
        out = self.rl(out)
        
        return out

In [10]:
''' Memo : CNN size equation (no dilation)

                                OUT = (IN + 2*Padding - Kernel_size) / Stride + 1                 
'''
# define Inpainter (Generator for GAN)
# U-net architecture
class Inpainter(nn.Module):
    def __init__(self):
        super(Inpainter, self).__init__()
        
        # U-net encoder
        # default: kernel_size = 4, stride = 2, padding = 1, using batchnorm, no dropout
        self.encoder1 = Downsample(  3,  32, use_batchnorm = False)   # out tensor size: (batchsize,  32, 112, 112)
        self.encoder2 = Downsample( 32,  64)                          # out tensor size: (batchsize,  64,  56,  56)
        self.encoder3 = Downsample( 64, 128)                          # out tensor size: (batchsize, 128,  28,  28)
        self.encoder4 = Downsample(128, 256)                          # out tensor size: (batchsize, 256,  14,  14)
        self.encoder5 = Downsample(256, 512)                          # out tensor size: (batchsize, 512,   7,   7)
        
        # U-net decoder
        # default: kernel_size = 4, stride = 2, padding = 1, using batchnorm, no dropout
        self.decoder1 = Upsample(512    , 512)                        # out tensor size: (batchsize, 512,  14,  14)
        self.decoder2 = Upsample(512+256, 512)                        # out tensor size: (batchsize, 512,  28,  28)
        self.decoder3 = Upsample(512+128, 256)                        # out tensor size: (batchsize, 256,  56,  56)
        self.decoder4 = Upsample(256+ 64, 128)                        # out tensor size: (batchsize, 128, 112, 112)
        self.decoder5 = Upsample(128+ 32,  64)                        # out tensor size: (batchsize,  64, 224, 224)
        
        # pointwise convolution to adjust channel with no image size change
        self.decoder_final = nn.Conv2d(64, 3, kernel_size = 1, stride = 1, padding = 0)
        self.th = nn.Tanh()
        
    def forward(self, x):
        # encoding part
        out_encoder1 = self.encoder1(x)
        out_encoder2 = self.encoder2(out_encoder1)
        out_encoder3 = self.encoder3(out_encoder2)
        out_encoder4 = self.encoder4(out_encoder3)
        out_encoder5 = self.encoder5(out_encoder4)
        
        # decording part
        out_decoder1 = self.decoder1(out_encoder5)
        out_decoder2 = self.decoder2(torch.cat([out_decoder1, out_encoder4], dim = 1))
        out_decoder3 = self.decoder3(torch.cat([out_decoder2, out_encoder3], dim = 1))
        out_decoder4 = self.decoder4(torch.cat([out_decoder3, out_encoder2], dim = 1))
        out_decoder5 = self.decoder5(torch.cat([out_decoder4, out_encoder1], dim = 1))
        
        out = self.decoder_final(out_decoder5)
        out = self.th(out)
        
        return out

In [17]:
''' Memo : CNN size equation (no dilation)

                                OUT = (IN + 2*Padding - Kernel_size) / Stride + 1                 
'''
# define Detector (Discriminator for GAN)
# FCN-architecture (PatchGAN discriminator)
''' Issue : Should we condition discriminator by input Image? '''
class Detector(nn.Module):
    def __init__(self):
        super(Detector, self).__init__()
        # default: kernel_size = 4, stride = 2, padding = 1, using batchnorm, no dropout
        self.fcn1x = Downsample(  3,  32, use_batchnorm = False)   # out tensor size: (batchsize,  32, 112, 112)
        self.fcn1y = Downsample(  3,  32, use_batchnorm = False)   # out tensor size: (batchsize,  32, 112, 112)
        self.fcn2  = Downsample( 64,  64)                          # out tensor size: (batchsize,  64,  56,  56)
        self.fcn3  = Downsample( 64, 128)                          # out tensor size: (batchsize, 128,  28,  28)
        self.fcn4  = Downsample(128, 256)                          # out tensor size: (batchsize, 256,  14,  14)
        self.fcn5  = Downsample(256, 512, stride = 1, padding = 0) # out tensor size: (batchsize, 512,  11,  11)
        
        # pointwise convolution to adjust channel with no image size change
        self.fcn_final = nn.Conv2d(512, 1, kernel_size = 1, stride = 1, padding = 0)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, y):
        # input x:Inpainter output, y:original image for conditioning
        out_x = self.fcn1x(x)
        out_y = self.fcn1y(y)
        out = torch.cat([out_x, out_y], dim = 1)
        
        out = self.fcn2(out)
        out = self.fcn3(out)
        out = self.fcn4(out)
        out = self.fcn5(out)
        
        out = self.fcn_final(out)
        out = self.sigmoid(out)
        
        return out

In [33]:
# network, optimizer and hyperparameters settings

# instantiate networks
inpainter = Inpainter()
detector = Detector()

# send to GPU(CPU)
inpainter = inpainter.to(device)
detector = detector.to(device)

# set optimizer
inp_optimizer = optim.Adam(inpainter.parameters(), lr = 0.0002, betas = [0.5, 0.999])
det_optimizer = optim.Adam(detector.parameters(), lr = 0.0002, betas = [0.5, 0.999])

# init weights
for p in inpainter.parameters():
    nn.init.normal_(p, mean = 0, std = 0.02)
    
for p in detector.parameters():
    nn.init.normal_(p, mean = 0, std = 0.02)

# count the number of trainable parameters
num_trainable_params_inp = sum(p.numel() for p in inpainter.parameters() if p.requires_grad)
num_trainable_params_det = sum(p.numel() for p in detector.parameters() if p.requires_grad)

# coefficient for gaussian noise to add to inpainter input
γ = 0.4

In [34]:
# print settings

print('--------------------------------------------------------------------------------------')
print('Inpainter')
print('The number of trainable parameters:', num_trainable_params_inp)
print('\nModel\n', inpainter)
print('\nOptimizer\n', inp_optimizer)

print('--------------------------------------------------------------------------------------')
print('Detector')
print('The number of trainable parameters:', num_trainable_params_det)
print('\nModel\n', detector)
print('\nOptimizer\n', det_optimizer)

--------------------------------------------------------------------------------------
Inpainter
The number of trainable parameters: 16720803

Model
 Inpainter(
  (encoder1): Downsample(
    (cv): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (dr): Dropout(p=0.5)
    (rl): LeakyReLU(negative_slope=0.2)
  )
  (encoder2): Downsample(
    (cv): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (dr): Dropout(p=0.5)
    (rl): LeakyReLU(negative_slope=0.2)
  )
  (encoder3): Downsample(
    (cv): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (dr): Dropout(p=0.5)
    (rl): LeakyReLU(negative_slope=0.2)
  )
  (encoder4): Downsample(
    (cv): Conv2d(128, 256, ke

In [35]:
# define loss function
def inp_loss(det_output_fake):
    adversarial_loss = F.binary_cross_entropy(det_output_fake, torch.ones_like(det_output_fake))
    return adversarial_loss

def det_loss(det_output_real, det_output_fake):
    real_loss = F.binary_cross_entropy(det_output_real, torch.ones_like(det_output_real))
    fake_loss = F.binary_cross_entropy(det_output_fake, torch.zeros_like(det_output_fake))
    adversarial_loss = 0.5 * (real_loss + fake_loss) # devide by 2 to regulate learning speed (mentioned by pix2pix)
    return adversarial_loss

In [47]:
def train(data_loader):
    inpainter.train()
    detector.train()
    
    running_inpainter_loss = 0
    running_detector_loss = 0
    
    for inputs in data_loader:
        inputs = inputs.to(device)
        # make contaminated inputs for inpainter network by adding gaussian noise
        inputs_contaminated = inputs + γ * torch.randn(inputs.size()).to(device)
        
        # calculate network outputs
        inp_outputs = inpainter(inputs_contaminated)
        det_outputs_real = detector(inputs, inputs)
        # .detach() is used not to track inpainter gradient during detector backward calculation
        det_outputs_fake_for_det = detector(inp_outputs.detach(), inputs)
        det_outputs_fake_for_inp = detector(inp_outputs, inputs)
        
        # detector : calculate loss function, run backward calculation, and update weights
        det_optimizer.zero_grad()
        detector_loss = det_loss(det_outputs_real, det_outputs_fake_for_det)
        detector_loss.backward()
        det_optimizer.step()
        
        # inpainter : calculate loss function, run backward calculation, and update weights
        inp_optimizer.zero_grad()
        inpainter_loss = inp_loss(det_outputs_fake_for_inp)
        inpainter_loss.backward()
        inp_optimizer.step()
        
        # extract number from zero dimension tensor by .item()
        running_inpainter_loss += inpainter_loss.item()
        running_detector_loss += detector_loss.item()
        
    # devide by len(data_loader) because F.binary_cross_entropy normalize loss in minibatch
    average_inpainter_loss = running_inpainter_loss / len(data_loader)
    average_detector_loss = running_detector_loss / len(data_loader)
    
    return average_inpainter_loss, average_detector_loss

In [66]:
n_save_images = 5
interval_save_images = 5 # epoch

def validation(data_loader, epoch):
    inpainter.train()
    detector.train()
    
    running_inpainter_loss = 0
    running_detector_loss = 0
    
    with torch.no_grad():
        for inputs in data_loader:
            inputs = inputs.to(device)
            inputs_contaminated = inputs + γ * torch.randn(inputs.size()).to(device)

            # calculate network outputs
            # some code that needn't for validation is deleted from train()
            inp_outputs = inpainter(inputs_contaminated)
            det_outputs_real = detector(inputs, inputs)
            det_outputs_fake = detector(inp_outputs, inputs)
            
            running_detector_loss += det_loss(det_outputs_real, det_outputs_fake).item()
            running_inpainter_loss += inp_loss(det_outputs_fake).item()
            
    # save [n_save_images] (normal image, contaminated image, inpainted image) comparison image
    if epoch % interval_save_images == 0:
        for n in range(n_save_images):
            normal_image = inputs[n].unsqueeze(0)
            contaminated_image = inputs_contaminated[n].unsqueeze(0)
            inpainted_image = inp_outputs[n].unsqueeze(0)
            comparison = torch.cat([normal_image, contaminated_image, inpainted_image])
            save_image(comparison.data.cpu(), '{}/{}_{}.png'.format(output_dir, epoch, n))
                    
    average_inpainter_loss = running_inpainter_loss / len(data_loader)
    average_detector_loss = running_detector_loss / len(data_loader)
    
    return average_inpainter_loss, average_detector_loss

In [ ]:
n_epochs = 3
train_loss_list = [[], []]
validation_loss_list = [[], []]

for epoch in range(n_epochs):
    train_inp_loss, train_det_loss = train(train_loader)
    validation_inp_loss, validation_det_loss = validation(validation_loader, epoch)
    
    train_loss_list[0].append(train_inp_loss)
    train_loss_list[1].append(train_det_loss)
    validation_loss_list[0].append(validation_inp_loss)
    validation_loss_list[1].append(validation_det_loss)
    
    print('epoch[%d/%d] losses[train_inp:%1.4f train_det:%1.4f  validation_inp:%1.4f validation_det:%1.4f]' \
                        % (epoch+1, n_epochs, train_inp_loss, train_det_loss, validation_inp_loss, validation_det_loss)) 


# save state_dicts
torch.save(inpainter.state_dict(), save_dir + 'inpainter_' + str(epoch) + '.pth')
torch.save(detector.state_dict(), save_dir + 'detector_' + str(epoch) + '.pth')
torch.save(inp_optimizer.state_dict(), save_dir + 'inp_optmizer_' + str(epoch) + '.pth')
torch.save(det_optimizer.state_dict(), save_dir + 'det_discriminator' + str(epoch) + '.pth')

# save learning log
np.save(save_dir + 'train_loss_list.npy', np.array(train_loss_list))
np.save(save_dir + 'validation_loss_list.npy', np.array(validation_loss_list))